In [44]:
!pip install -q pyomo

In [45]:
from pyomo.environ import *

In [46]:
import numpy as np

In [47]:
model = ConcreteModel()

In [48]:
N = 12
M = 5

In [49]:
obj_coef_c = np.array([0.56,1.721,-9.6,-2.85,41.5,33.05,-0.75,20.5,-0.35,-31.02,50.65,6.125])

In [50]:
constr_coef_A = np.array([[1,-1,0,0,2,-2,1,0,0,-1,1,-2],[1,0,0,-1,0,-2,0,-1,1,0,0,-1],[-1,0,0,1,-1,2.5,-3.5,0,2,-1,1,0],[0,1,-0.45,0,-2.78,0,-3.4,0,-1,1,-1,0.25],[0,-1,1,0,1,0,1,3,-2,0,1,0]])

constr_rhs_b = 88.5,35.4,47.9,41.43,22.7

In [51]:
lower_bound = np.array([0,0,-np.inf,-np.inf,-34,0,-67,0,-23,-5,6,1])
upper_bound = np.array([12,503,4,+np.inf,+np.inf,2,20,55,23,15,+np.inf,+np.inf])

In [52]:
col_indices = np.arange(N)
print(col_indices)
print(col_indices[11])

[ 0  1  2  3  4  5  6  7  8  9 10 11]
11


In [53]:
row_indices = np.arange(M)
print(row_indices)
print(row_indices[4])


[0 1 2 3 4]
4


In [54]:
model.x = Var(col_indices)

In [55]:
model.constraints = ConstraintList()


In [56]:
row_indices = np.arange(M)

In [57]:
for i in row_indices:
  model.constraints.add(sum(constr_coef_A[i][j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])

In [58]:
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [59]:
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=minimize)

In [60]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   0.0 :  None :  12.0 : False :  True :  Reals
          1 :   0.0 :  None : 503.0 : False :  True :  Reals
          2 :  None :  None :   4.0 : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 : -34.0 :  None :  None : False :  True :  Reals
          5 :   0.0 :  None :   2.0 : False :  True :  Reals
          6 : -67.0 :  None :  20.0 : False :  True :  Reals
          7 :   0.0 :  None :  55.0 : False :  True :  Reals
          8 : -23.0 :  None :  23.0 : False 

In [61]:
!apt-get install -y -qq glpk-utils

In [62]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -1951.831
  Upper bound: -1951.831
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 13
  Number of nonzeros: 38
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.014335870742797852
# ----------------------------------------------------------
#   Solution Information
# --------------------------

**Exercise 2,Ans(1):**
The solution of the problem is shown above, and the optimal values of decision variables and the value of the optimal objective function is shown in the below cell;

In [63]:
print('\nObjective = ', model.objective())
print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  -1951.8309999999997

Decision Variables
x[ 0 ] =  12.0
x[ 1 ] =  0.0
x[ 2 ] =  4.0
x[ 3 ] =  119.82
x[ 4 ] =  -34.0
x[ 5 ] =  0.0
x[ 6 ] =  20.0
x[ 7 ] =  0.0
x[ 8 ] =  -7.46000000000002
x[ 9 ] =  15.0
x[ 10 ] =  6.0
x[ 11 ] =  1.0

Constraints
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :               -47.0 :  88.5
      2 :  None : -116.28000000000002 :  35.4
      3 :  None :   47.89999999999995 :  47.9
      4 :  None :   41.43000000000002 : 41.43
      5 :  None :   10.92000000000004 :  22.7


**Exercise 2,Ans(2):**
The nature of constraints of a coefficient matrix are as follows;

*Constraint 1(1st row)* is inactive because the value is not equal to RHS.

*Constraint 2(2nd row)* is inactive because the value is not equal to RHS.

*Constraint 3(3rd row)* is active because the value is equal to RHS.

*Constraint 4(4th row)* is active because the value is equal to RHS.

*Constraint 5(5th row)* is inactive because the value is not equal to RHS.
